In [1]:
print("hi")

hi


In [2]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def load_dataset(root_dir):
    """
    Recursively parse the LGG MRI dataset.
    Returns a list of dictionaries:
    [
      {
        "image_path": "...",
        "mask_path": "...",
        "image": np.ndarray,
        "mask": np.ndarray
      },
      ...
    ]
    """
    data = []

    # Walk through all subdirectories
    for subdir, dirs, files in os.walk(root_dir):
        # Only consider tif files
        tifs = [f for f in files if f.endswith(".tif")]

        # Create mapping: index -> file
        image_files = {f.replace("_mask", ""): f for f in tifs if "_mask" not in f}
        mask_files  = {f.replace("_mask", ""): f for f in tifs if "_mask" in f}

        # Pair image + mask files
        for key in sorted(image_files.keys()):
            if key in mask_files:
                img_path = os.path.join(subdir, image_files[key])
                mask_path = os.path.join(subdir, mask_files[key])

                # Load arrays
                img = np.array(Image.open(img_path))
                mask = np.array(Image.open(mask_path))

                data.append({
                    "image_path": img_path,
                    "mask_path": mask_path,
                    "image": img,
                    "mask": mask
                })

    return data


if __name__ == "__main__":
    root = "/home/tyler/projects/UNet/data/lgg-mri-segmentation/kaggle_3m"
    dataset = load_dataset(root)

    print(f"Loaded {len(dataset)} image/mask pairs.\n")

    # Example: print one sample
    sample = dataset[0]
    print("Image:", sample["image_path"])
    print("Mask:", sample["mask_path"])
    print("Image shape:", sample["image"].shape)
    print("Mask shape:", sample["mask"].shape)

    # ---- Split into tumor / non-tumor ----
    tumor = []
    no_tumor = []
    
    for item in dataset:
        if np.any(item["mask"] > 0):
            tumor.append(item)
        else:
            no_tumor.append(item)
    
    print("Tumor images:", len(tumor))
    print("No-tumor images:", len(no_tumor))
    
    images = np.stack([
        np.transpose(item["image"], (2, 0, 1))  # (H,W,C) → (C,H,W)
        for item in dataset
    ]).astype(np.float32)                        # final shape [N, 3, 256, 256]

    masks = np.stack([
        item["mask"]                             # (H,W)
        for item in dataset
    ]).astype(np.float32)                          # final shape [N, 256, 256]
    images = np.ascontiguousarray(images)
    masks = np.ascontiguousarray(masks)
    N, C, H, W = images.shape
    shape = [float(N), float(C), float(H), float(W)]



Loaded 3929 image/mask pairs.

Image: /home/tyler/projects/UNet/data/lgg-mri-segmentation/kaggle_3m/TCGA_DU_8164_19970111/TCGA_DU_8164_19970111_1.tif
Mask: /home/tyler/projects/UNet/data/lgg-mri-segmentation/kaggle_3m/TCGA_DU_8164_19970111/TCGA_DU_8164_19970111_1_mask.tif
Image shape: (256, 256, 3)
Mask shape: (256, 256)
Tumor images: 1373
No-tumor images: 2556


In [3]:
from neural_net import NeuralNet
from neural_net import LayerType
from neural_net import LayerDesc

ImportError: /home/tyler/projects/UNet/.venv/lib/python3.12/site-packages/neural_net.cpython-312-x86_64-linux-gnu.so: undefined symbol: _ZN5Layer19forward_convolutionEPfib

In [ ]:
in_height = 256
in_width = 256
in_depth = 3

UNET = NeuralNet()   # instantiate the C++ class
UNET.create([
    LayerDesc(LayerType.CONV_LAYER,[in_height, in_width, in_depth, 1, 64, 3,1,1], []), #0
    LayerDesc(LayerType.MAX_POOL_LAYER, [2,2],[0]), #1
    LayerDesc(LayerType.CONV_LAYER, [in_height//2, in_width//2, in_depth, 64, 128, 3,1,1],[1]),#2
    LayerDesc(LayerType.MAX_POOL_LAYER, [2,2],[2]), #3
    LayerDesc(LayerType.CONV_LAYER, [in_height//4, in_width//4, in_depth, 128, 256, 3,1,1],[3]), #4
    LayerDesc(LayerType.MAX_POOL_LAYER, [2,2],[4]), #5
    LayerDesc(LayerType.CONV_LAYER, [in_height//8, in_width//8, in_depth,256, 512, 3,1,1],[5]), #6
    #parameters are h_in, w_in, c_in, c_out, stride/upscale
    LayerDesc(LayerType.UPSAMPLING_LAYER, [in_height//8, in_width//8, 512, 256, 2],[6]), #7
    #parameters are fully determined by parents. First parent is skip connection, second is convolution below 
    LayerDesc(LayerType.ATTENTION_LAYER, [],[4,6]), #8
    #parameters are fully determined by parents. First parent is from skip/attention, second is from convolution below
    LayerDesc(LayerType.CONCAT_LAYER, [],[8,7]), #9
    LayerDesc(LayerType.CONV_LAYER, [in_height//4, in_width//4, in_depth,512, 128, 3,1,1],[9]), #10
    LayerDesc(LayerType.UPSAMPLING_LAYER, [in_height//4, in_width//4, 128, 128, 2],[10]), #11
    LayerDesc(LayerType.ATTENTION_LAYER, [],[2,10]), #12 
    LayerDesc(LayerType.CONCAT_LAYER, [],[12,11]), #13
    LayerDesc(LayerType.CONV_LAYER, [in_height//2, in_width//2, in_depth,256, 64, 3,1,1],[13]), #14
    LayerDesc(LayerType.UPSAMPLING_LAYER, [in_height//2, in_width//2, 64, 64, 2],[14]), #15
    LayerDesc(LayerType.ATTENTION_LAYER, [],[0,14]), #16 
    LayerDesc(LayerType.CONCAT_LAYER, [],[16,15]), #17
    LayerDesc(LayerType.CONV_LAYER, [in_height, in_width, in_depth,128, 1, 3,1,1],[17]), #18
])


learning_rate = .001
epochs = 10
batch_size = 16
UNET.train(images,shape,masks,learning_rate, epochs, batch_size)


recieved images
3


In [ ]:
print(dataset[2]["image"].shape)
print(dataset[2]["image"][40][30][1])
print(len(dataset)/16)


(256, 256, 3)
3
245.5625
